# VLM Benchmark for Object Property Abstraction

This notebook implements a benchmark for evaluating Vision Language Models (VLMs) on object property abstraction and visual question answering (VQA) tasks. The benchmark includes three types of questions:

1. Direct Recognition
2. Property Inference
3. Counterfactual Reasoning

And three types of images:
- REAL
- ANIMATED
- AI GENERATED

## Setup and Imports

First, let's import the necessary libraries and set up our environment.

In [ ]:
# %pip install qwen-vl-utils flash-attn #--no-build-isolation
%pip install -q -U google-genai

In [3]:
# Import required libraries
import torch
import os
import json
from pathlib import Path
from PIL import Image
import gc
import re
from tqdm import tqdm
from typing import List, Dict, Any
from google import genai
from dotenv import load_dotenv

load_dotenv()

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

/var/scratch/ave303/anaconda3/envs/op_bench/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [ ]:
# Get API key from environment
api_key = os.getenv('GOOGLE_API_KEY')
if not api_key:
    raise ValueError("GOOGLE_API_KEY not found in environment")

## Benchmark Tester Class

This class handles the evaluation of models against our benchmark.

In [4]:
class BenchmarkTester:
    def __init__(self, benchmark_path="/var/scratch/ave303/OP_bench/benchmark.json", data_dir="/var/scratch/ave303/OP_bench/"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        with open(benchmark_path, 'r') as f:
            self.benchmark = json.load(f)
        self.data_dir = data_dir
        self.api_key = os.getenv('GOOGLE_API_KEY')

    def clean_answer(self, answer):
        """Extract number and reasoning from the model's answer."""
        # Try to extract number and reasoning using regex
        import re
        pattern = r'(\d+)\s*\[(.*?)\]'
        match = re.search(pattern, answer)
        
        if match:
            number = match.group(1)
            objects = [obj.strip() for obj in match.group(2).split(',')]
            return {
                "count": number,
                "reasoning": objects
            }
        else:
            # Fallback if format isn't matched
            numbers = re.findall(r'\d+', answer)
            return {
                "count": numbers[0] if numbers else "0",
                "reasoning": []
            }
    
    def evaluate_model(self, model_name, save_path, start_idx=0, batch_size=5):
        results = []
        print(f"\nEvaluating {model_name}...")
        print(f"Using device: {self.device}")
        
        # client = genai.Client(api_key=self.api_key)

        # Force garbage collection before starting
        gc.collect()
        torch.cuda.empty_cache()

        try:
            images = self.benchmark['benchmark']['images'][start_idx:start_idx + batch_size]
            total_images = len(images)
            
            for idx, image_data in enumerate(tqdm(images, desc="Processing images")):
                try:
                    client = genai.Client(api_key=self.api_key)
                    print(f"\nProcessing image {idx+1}/{total_images}: {image_data['image_id']}")
                    image_path = Path(self.data_dir)/image_data['path']
                    if not image_path.exists():
                        print(f"Warning: Image not found at {image_path}")
                        continue
                    
                    # Load and preprocess image
                    # image = Image.open(image_path).convert("RGB")
                    
                    image = client.files.upload(file=image_path)
                    image_results = []  # Store results for current image
                    
                    for question in image_data['questions']:
                        try:
                            print(f"Question: {question['question']}")

                            response = client.models.generate_content(
                                model=model_name,
                                contents=[image, f"{question['question']} Your response MUST be in the following format and nothing else:\n <NUMBER> [<OBJECT1>, <OBJECT2>, <OBJECT3>, ...]"],
                            )
                            # print(response.text)
                            
                            # Clear cache before processing each question
                            torch.cuda.empty_cache()
                            
                            cleaned_answer = self.clean_answer(response.text)
                            
                            image_results.append({
                                "image_id": image_data["image_id"],
                                "image_type": image_data["image_type"],
                                "question_id": question["id"],
                                "question": question["question"],
                                "ground_truth": question["answer"],
                                "model_answer": cleaned_answer["count"],
                                "model_reasoning": cleaned_answer["reasoning"],
                                "raw_answer": response.text,  # Keep raw answer for debugging
                                "property_category": question["property_category"]
                            })
                            
                            # # Clear memory
                            # del outputs, inputs
                            # torch.cuda.empty_cache()
                            
                        except Exception as e:
                            print(f"Error processing question: {str(e)}")
                            continue
                    
                    # Add results from this image
                    results.extend(image_results)

                    client.files.delete(name=image.name)
                            
                except Exception as e:
                    print(f"Error processing image {image_data['image_id']}: {str(e)}")
                    continue
            
            # Save final results
            if results:
                with open(save_path, 'w') as f:
                    json.dump(results, f, indent=4)
            
        except Exception as e:
            print(f"An error occurred during evaluation: {str(e)}")
            if results:
                with open(f"{save_path}_error_state.json", 'w') as f:
                    json.dump(results, f, indent=4)
        
        return results

## Test Gemini 2.0 Flash

Let's evaluate Gemini-2.0-flash model

In [ ]:
def test_Gemini2_0Flash():
    tester = BenchmarkTester()
    Gemini2_0Flash_results = tester.evaluate_model(
        "gemini-2.0-flash",
        "Gemini2_0Flash_results.json",
        start_idx=45,
        batch_size=5
    )

## Run Evaluation

Now we can run our evaluation. Let's start with the Gemini-2.0-flash model:

In [ ]:
test_Gemini2_0Flash()